In [7]:
import pandas as pd
import fiona
from rdflib import Graph, Namespace, URIRef, XSD, Literal

Dataset of population of Austrian municipalities: 

https://www.statistik.at/statistiken/bevoelkerung-und-soziales/bevoelkerung/bevoelkerungsstand/bevoelkerung-zu-jahres-/-quartalsanfang

In [8]:
# import population data Austria
population_data = pd.read_csv("/mnt/data/Population/bevölkerung_gemeinden_österreich.csv", skiprows=1, delimiter=";", encoding="utf-8")

In [9]:
population_data.rename(columns={"Unnamed: 1": "Gemeindename"}, inplace=True)

In [10]:
year_of_interest = "2021"
population_select = population_data.loc[:, ["Gemeindename", year_of_interest]]
population_select[year_of_interest] = population_select[year_of_interest].apply(lambda x: x.replace(" ", ""))
population_select[year_of_interest] = population_select[year_of_interest].apply(pd.to_numeric)

In [11]:
population_select.head()

,Gemeindename,2021
0,ÖSTERREICH,8932664
1,Eisenstadt,14895
2,Rust,2000
3,Breitenbrunn am Neusiedler See,1883
4,Donnerskirchen,1841


In [12]:
# load gadm dataset 
path_to_gadm_graph = "/mnt/data/GADM/RDF/gadm_AUT_RDF.ttl"

gadm = Namespace("http://example.com/ontologies/gadm#")

graph = Graph()
graph.bind("gadm", gadm)

graph.parse(path_to_gadm_graph, format='ttl')

<Graph identifier=N355c1ec265ea4e36a202e7f538683794 (<class 'rdflib.graph.Graph'>)>

In [13]:
ent_hasPop = URIRef(gadm["hasPopulation2021"])

In [14]:
query = """
SELECT ?subject ?name
WHERE {
  ?subject gadm:hasNationalLevel 3 ;
           gadm:hasName ?name .
}
"""

# Execute the query and iterate over the results
for row in graph.query(query):
    admin_ent, name = row
    try:
        pop = population_select[population_select.Gemeindename == name.strip()]["2021"].values[0]
        graph.add((admin_ent, ent_hasPop, Literal(pop, datatype=XSD.integer))) 
    except:
        pass

In [15]:
graph.serialize("/mnt/data/GADM/RDF/gadm_AUT_RDF_population.ttl")

<Graph identifier=N355c1ec265ea4e36a202e7f538683794 (<class 'rdflib.graph.Graph'>)>